In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import datetime

import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, LSTM, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import keras_tuner
from keras import regularizers

2024-03-24 12:38:01.791300: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 12:38:01.793676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 12:38:01.836597: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 12:38:01.836619: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 12:38:01.836640: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self,hp):
        model = Sequential()
        model.add(InputLayer(input_shape=(240,1 )))
        hp_units = hp.Choice('units', values=[32,64,128])
        hp_lr = hp.Choice('learning_rate', values=[1e-6,1e-7,1e-8])
        hp_regulariers = hp.Choice('regularies', values = ['l1','l2', 'None'])
        if hp_regulariers == 'l1':
            regularizers = keras.regularizers.L1(hp_lr)
        elif hp_regulariers == 'l2':
            regularizers = keras.regularizers.L2(hp_lr)
        elif hp_regulariers == 'None':
            regularizers = None
        
        model.add(LSTM(units=hp_units,kernel_regularizer=regularizers,name='dense_0'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        
        hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam',"adamax"])
        if hp_optimizer == 'sgd':
            optimizer = keras.optimizers.SGD(learning_rate=hp_lr)
        elif hp_optimizer == 'rmsprop':
            optimizer = keras.optimizers.RMSprop(learning_rate=hp_lr)
        elif hp_optimizer == 'adam':
            optimizer = keras.optimizers.Adam(learning_rate=hp_lr)
        elif hp_optimizer == 'adamax':
            optimizer = keras.optimizers.Adamax(learning_rate=hp_lr)
        else:
            raise ValueError("Invalid optimizer choice")

        model.compile(
            optimizer = optimizer
            ,loss=keras.losses.BinaryCrossentropy()
            , metrics=['accuracy'])
        return model


    def fit(self, hp, model, *args, **kwargs):
            return model.fit(
                *args,
                batch_size=hp.Choice("batch_size", [16,32,64,128]),
                **kwargs,
            )

In [4]:
timesteps = 240
num_input =1
num_classes=1
label = list(range(timesteps)) + ['target'] + ['ticker'] + ['target_date'] + ['sector']

training_data = []
training_label = []
testing_data =[]
testing_label =[]

accuracy_results = []


for i in range(5):
    # read the data
    path  = '/home/RDC/yeungwin/H:/yeungwin/SP500/data/'
    train = pd.read_csv(path+'Set_' + str(i) + '_Train.csv', index_col=0).dropna()
    test = pd.read_csv(path+'Set_' + str(i) + '_Test.csv', index_col=0).dropna()

    train.columns = label
    test.columns = label

    train_label = train.iloc[:, timesteps]
    train_data = train.iloc[:, :timesteps]
    test_label = test.iloc[:,timesteps]
    test_data = test.iloc[:, :timesteps]
    
    
     # reshape input
    #  data: (samples, timesteps, features)
    x_train = np.array(train_data).reshape((len(train_data), timesteps, num_input), order = 'F')
    x_test = np.array(test_data).reshape((len(test_data), timesteps, num_input), order = 'F')
    # label: (samples, target)
    y_train = np.array(train_label).reshape((len(train_label), num_classes))
    y_test = np.array(test_label).reshape((len(test_label), num_classes))
        
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    
    print("-------------------------------------------------------------------------------------------------------")
    print("Training the model for Training Set " + str(i) + " from " +
    datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
    print("-------------------------------------------------------------------------------------------------------")
        
    if i ==0:   
        tuner = keras_tuner.BayesianOptimization(MyHyperModel(),
            objective='val_accuracy',# overwrite=True,
            max_trials=30, directory='lstm', seed=4)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience =10, restore_best_weights=False)
        tuner.search(x_train,y_train, epochs =1000, validation_split=0.2, callbacks=[early_stop])

        # save the best model
        hypermodel = MyHyperModel()
        best_hp = tuner.get_best_hyperparameters()[0]
        best_model = hypermodel.build(best_hp)

        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 10, restore_best_weights=False)
        result = best_model.fit(x_train,y_train, epochs=1000, validation_split =0.2, verbose =1, callbacks=[early_stop])
        print(tuner.get_best_hyperparameters()[0].get_config()["values"])

    else: 
        load_path = '/home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight.h5'
        print('Model restore from ' + load_path)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=load_path,
                                                 save_weights_only=True,
                                                 verbose=1)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                    patience = 10, restore_best_weights=False)

        result = best_model.fit(
            x_train, 
            y_train, 
            epochs = 1000, 
            validation_split=0.2,
            verbose =1,
            callbacks=[cp_callback, early_stop]        
        ) 
            
    model_path ="/home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight.h5"    
    best_model.save_weights(model_path)
    print("Model saved to " + model_path)
    print("Training end: " + datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
        
    ##make prediction
    pred_ff_test = best_model.predict(x_test)
    #pred = pred_ff_test.tolist()
    pred = pred_ff_test.reshape((1, len(pred_ff_test))).tolist()[0]
    output_data = pd.DataFrame({'y_prob': pred, 'y_true': test['target'], 'Ticker': test['ticker'],
                                    'Date': test['target_date'], 'Sector': test['sector'], })
    accuracy = accuracy_score(np.round(output_data['y_prob']), output_data['y_true'])
    print('Overall Accuracy for test set:'+ str(accuracy))
    output_data.to_csv('/home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/pred10/lstm_prediction_period_'+str(i)+'.csv')
    print('Prediction for period ' + str(i) + ' successfully saved.')

(245464, 240, 1)
(245464, 1)
(120328, 240, 1)
(120328, 1)
-------------------------------------------------------------------------------------------------------
Training the model for Training Set 0 from 2024-03-24 12:38:35
-------------------------------------------------------------------------------------------------------
Reloading Tuner from lstm10/untitled_project/tuner0.json
Epoch 1/1000
6137/6137 [==============================] - 544s 88ms/step - loss: 0.6934 - accuracy: 0.5015 - val_loss: 0.6931 - val_accuracy: 0.5010
Epoch 2/1000
6137/6137 [==============================] - 542s 88ms/step - loss: 0.6934 - accuracy: 0.5004 - val_loss: 0.6931 - val_accuracy: 0.5027
Epoch 3/1000
6137/6137 [==============================] - 543s 88ms/step - loss: 0.6935 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.5051
Epoch 4/1000
6137/6137 [==============================] - 541s 88ms/step - loss: 0.6933 - accuracy: 0.5018 - val_loss: 0.6931 - val_accuracy: 0.5056
Epoch 5/1000
6137/

6137/6137 [==============================] - 589s 96ms/step - loss: 0.6933 - accuracy: 0.5053 - val_loss: 0.6930 - val_accuracy: 0.5096
Epoch 108/1000
6137/6137 [==============================] - 589s 96ms/step - loss: 0.6932 - accuracy: 0.5059 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 109/1000
6137/6137 [==============================] - 589s 96ms/step - loss: 0.6932 - accuracy: 0.5071 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 110/1000
6137/6137 [==============================] - 590s 96ms/step - loss: 0.6932 - accuracy: 0.5071 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 111/1000
6137/6137 [==============================] - 585s 95ms/step - loss: 0.6932 - accuracy: 0.5064 - val_loss: 0.6930 - val_accuracy: 0.5098
Epoch 112/1000
6137/6137 [==============================] - 594s 97ms/step - loss: 0.6932 - accuracy: 0.5071 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 113/1000
6137/6137 [==============================] - 587s 96ms/step - loss: 0.6932 - accuracy: 0.5067 

6137/6137 [==============================] - 591s 96ms/step - loss: 0.6931 - accuracy: 0.5079 - val_loss: 0.6929 - val_accuracy: 0.5112
Epoch 216/1000
6137/6137 [==============================] - 589s 96ms/step - loss: 0.6931 - accuracy: 0.5077 - val_loss: 0.6929 - val_accuracy: 0.5114
Epoch 217/1000
6137/6137 [==============================] - 582s 95ms/step - loss: 0.6931 - accuracy: 0.5067 - val_loss: 0.6929 - val_accuracy: 0.5112
Epoch 218/1000
6137/6137 [==============================] - 588s 96ms/step - loss: 0.6932 - accuracy: 0.5064 - val_loss: 0.6929 - val_accuracy: 0.5112
Epoch 219/1000
6137/6137 [==============================] - 591s 96ms/step - loss: 0.6931 - accuracy: 0.5087 - val_loss: 0.6929 - val_accuracy: 0.5112
Epoch 220/1000
6137/6137 [==============================] - 590s 96ms/step - loss: 0.6932 - accuracy: 0.5071 - val_loss: 0.6929 - val_accuracy: 0.5111
Epoch 221/1000
6137/6137 [==============================] - 588s 96ms/step - loss: 0.6931 - accuracy: 0.5076 

6137/6137 [==============================] - 552s 90ms/step - loss: 0.6930 - accuracy: 0.5088 - val_loss: 0.6929 - val_accuracy: 0.5116
Epoch 324/1000
6137/6137 [==============================] - 543s 89ms/step - loss: 0.6930 - accuracy: 0.5086 - val_loss: 0.6929 - val_accuracy: 0.5116
Epoch 325/1000
6137/6137 [==============================] - 551s 90ms/step - loss: 0.6930 - accuracy: 0.5085 - val_loss: 0.6929 - val_accuracy: 0.5118
Epoch 326/1000
6137/6137 [==============================] - 549s 89ms/step - loss: 0.6930 - accuracy: 0.5085 - val_loss: 0.6929 - val_accuracy: 0.5118
Epoch 327/1000
6137/6137 [==============================] - 554s 90ms/step - loss: 0.6930 - accuracy: 0.5091 - val_loss: 0.6929 - val_accuracy: 0.5118
Epoch 328/1000
6137/6137 [==============================] - 551s 90ms/step - loss: 0.6930 - accuracy: 0.5088 - val_loss: 0.6929 - val_accuracy: 0.5116
Epoch 329/1000
6137/6137 [==============================] - 554s 90ms/step - loss: 0.6930 - accuracy: 0.5088 

6130/6130 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5068
Epoch 6: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6130/6130 [==============================] - 565s 92ms/step - loss: 0.6928 - accuracy: 0.5068 - val_loss: 0.6932 - val_accuracy: 0.5073
Epoch 7/1000
6130/6130 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5054
Epoch 7: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6130/6130 [==============================] - 561s 91ms/step - loss: 0.6928 - accuracy: 0.5054 - val_loss: 0.6932 - val_accuracy: 0.5067
Epoch 8/1000
6130/6130 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5054
Epoch 8: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6130/6130 [==============================] - 565s 92ms/step - loss: 0.6928 - accuracy: 0.5054 - val_loss: 0.6932 - val_accuracy: 0.5066
Epoch 9/100

...

Epoch 83/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5080
Epoch 83: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 589s 96ms/step - loss: 0.6923 - accuracy: 0.5080 - val_loss: 0.6905 - val_accuracy: 0.5064
Epoch 84/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5055
Epoch 84: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 595s 97ms/step - loss: 0.6923 - accuracy: 0.5055 - val_loss: 0.6905 - val_accuracy: 0.5064
Epoch 85/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5059
Epoch 85: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 590s 97ms/step - loss: 0.6923 - accuracy: 0.5059 - val_loss: 0.6905 - val_accuracy:

6110/6110 [==============================] - 551s 90ms/step - loss: 0.6923 - accuracy: 0.5059 - val_loss: 0.6904 - val_accuracy: 0.5061
Epoch 108/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5073
Epoch 108: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 560s 92ms/step - loss: 0.6923 - accuracy: 0.5073 - val_loss: 0.6904 - val_accuracy: 0.5061
Epoch 109/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5071
Epoch 109: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 556s 91ms/step - loss: 0.6922 - accuracy: 0.5071 - val_loss: 0.6904 - val_accuracy: 0.5060
Epoch 110/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5060
Epoch 110: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lst

Epoch 132/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6924 - accuracy: 0.5062
Epoch 132: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 546s 89ms/step - loss: 0.6924 - accuracy: 0.5062 - val_loss: 0.6904 - val_accuracy: 0.5053
Epoch 133/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5063
Epoch 133: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 541s 89ms/step - loss: 0.6923 - accuracy: 0.5063 - val_loss: 0.6904 - val_accuracy: 0.5049
Epoch 134/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5057
Epoch 134: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 568s 93ms/step - loss: 0.6923 - accuracy: 0.5057 - val_loss: 0.6904 - val_acc

6110/6110 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5061
Epoch 180: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 580s 95ms/step - loss: 0.6922 - accuracy: 0.5061 - val_loss: 0.6904 - val_accuracy: 0.5053
Epoch 181/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5071
Epoch 181: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 575s 94ms/step - loss: 0.6922 - accuracy: 0.5071 - val_loss: 0.6904 - val_accuracy: 0.5052
Epoch 182/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5072
Epoch 182: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 574s 94ms/step - loss: 0.6922 - accuracy: 0.5072 - val_loss: 0.6904 - val_accuracy: 0.5054
E

6110/6110 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5077
Epoch 228: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 544s 89ms/step - loss: 0.6923 - accuracy: 0.5077 - val_loss: 0.6904 - val_accuracy: 0.5057
Epoch 229/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5060
Epoch 229: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 571s 93ms/step - loss: 0.6922 - accuracy: 0.5060 - val_loss: 0.6904 - val_accuracy: 0.5058
Epoch 230/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5068
Epoch 230: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/5_LSTM/lstm_weight/lstm_weight10.h5
6110/6110 [==============================] - 574s 94ms/step - loss: 0.6922 - accuracy: 0.5068 - val_loss: 0.6904 - val_accuracy: 0.5058
E